<a href="https://colab.research.google.com/github/Elcejas77/Ejercicios/blob/main/Parcial2_Punto3_ArboledaDiego.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PARCIAL 2-PUNTO 3
DIEGO ALEJANDRO ARBOLEDA CUERO

CC:1087834596

Para la solucion de este punto utilizaremos como base el codigo guia del parcial,donde ya esta ingresada toda la informacion necesaria.

# Case Western Reserve Experiments


Sea la base de datos para el monitoreo de condición (fallos) en rodamientos a partir del análisis de vibraciones descrita en [Case Western Reserve Experiments](https://engineering.case.edu/bearingdatacenter). Las señales fueron adquiridas para las siguientes condiciones (clases): i) Normal bearing (Nor), fault in the internal train (IR1), fault in the external train (IR2), and fault in the rolling element-ball (BE). Además, los fallos se generaron para tres niveles de severidad (profundidad): 0.007′′, 0.014′′, and 0.021′′ y tres velocidades de operación (1730, 1750, 1772, and 1797 [rpm]). Los datos fueron adquiridos a 12 kHz. Por consiguiente, se tienen los siguientes parámetros de estudio: $F_s=12k$ [Hz] cantidad de puntos en el tiempo $4096$ y cantidad de clases $C = 10$.

Grafique la señal promedio de cada fallo en el tiempo y en la frecuencia.

Utilizando la transformada rápida de Fourier diseñe y construya un detector fallos en rodamientos a partir de señales de vibración y sus etiquetas en los arreglos Xtrain y Ytrain (ver cuaderno de apoyo). Genere las predicciones de fallos para el arreglo Xtest.

In [ ]:
#data downloaded for google drive
FILEID = "1IC11LrPCZIo_Am5eXP2p2tDAlrGTlPjn"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O datos.zip && rm -rf /tmp/cookies.txt
!unzip -o datos.zip
!dir

In [ ]:
#librerias
import scipy.io as sio
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import warnings
from sklearn.metrics import pairwise_distances
import matplotlib
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')

#cargar datos
path_ = 'CaractCE.mat'#Case Western Database
dicX = sio.loadmat(path_)

In [ ]:
Xt = dicX['F'] #datos en el tiempo
Fs = 12000 #frecuencia de muestreo
Tl = Xt.shape[1]/Fs #tamaño del segmento
print('Xt shape:',Xt.shape)
print('tiempo [s]', Tl)

Y = dicX['E']
Ytrue = Y[:,2] #clases fallos en los rodamientos

labels_ = ['NOR','IR1_0.007´´','IR1_0.014´´','IR1_0.021´´',
           'IR2_0.007´´','IR2_0.014´´','IR2_0.021´´',
           'BE_0.007´´','BE_0.014´´','BE_0.021´´'
           ] #nombres de las clases

In [ ]:
print(Ytrue.shape) #etique membresia de los datos 10 posibles valores
print(np.unique(Ytrue))

In [ ]:
#partir datos para train y test
Xtrain, Xtest, Ytrain, _ = train_test_split(Xt, Ytrue, test_size=0.3)

print(f"Xtrain shape {Xtrain.shape}, Ytrain shape {Ytrain.shape }Xtest shape {Xtest.shape} ")

In [ ]:
#calcular espectro de Fourier Xtrain
vf = np.fft.rfftfreq(Xtrain.shape[1],1/Fs) #freq vector
Xw = (abs(np.fft.rfft(Xtrain))) # FFT
Xw.shape

In [ ]:
#graficar espectro para clases representativas
sca_ = MinMaxScaler()
Xw_ = sca_.fit_transform(Xw.T).T
#red = TSNE(perplexity = 15,n_components=2,random_state=123,learning_rate='auto',init='pca')
red = PCA(n_components=2)
Z = red.fit_transform(Xw_)

plt.scatter(Z[:,0],Z[:,1],c=Ytrain, label='Xtrain')
plt.colorbar()
plt.show()

Tenemos que tener en cuenta que necesitamos una lista para almacenar los promedios de cada clase definida anteriormente

In [ ]:
import numpy as np

p_ti = []

# Calcular el promedio en el tiempo para cada clase
for clase in np.unique(Ytrue):
    p_cl = np.mean(Xtrain[Ytrain == clase], axis=0)
    p_ti.append(p_cl)



graficamos el pomedio en el tiempo de cada clase

In [ ]:
plt.figure(figsize=(5, 3))
for i, p_cl in enumerate(p_ti):
    plt.plot(p_cl, label=labels_[i])


plt.xlabel("Muestras en el tiempo")
plt.ylabel("Amplitud")
plt.legend()
plt.grid(True)
plt.show()

ya con la informacion obtenida dela base de datos podemos calcular lo9s promedios en frecuencia para cada clase y asi empezar a darle forma a la solucion del ejercicio

In [ ]:
#pr_f =promedio en frecuencia
#pr_espec=promedio espectro

pr_f = []  #vector donde estaran los valores del promedio en frecuencia

for clase in np.unique(Ytrain):
    espectros = Xw[Ytrain == clase]
    pr_espec = np.mean(espectros, axis=0)
    pr_f.append(pr_espec)


Se procede a graficar el promedio en frecuencia de cada clase obtenida en el codigo anterior

In [ ]:
plt.figure(figsize=(5, 3))
for i, promedio_espectro in enumerate(pr_f):
    plt.plot(vf, promedio_espectro, label=labels_[i])

plt.title("Promedio en la frecuencia de cada clase de fallo")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Amplitud")
plt.legend()
plt.grid(True)
plt.show()

calculamos la transformada para xwtest y le pedimos que nos devuelva la forma del vector

In [ ]:
vftest = np.fft.rfftfreq(Xtest.shape[1],1/Fs) #freq vector
Xwtest = (abs(np.fft.rfft(Xtest))) # FFT
Xwtest.shape

normalizamos el vector de amplitudes de componentes de frecuencia. y  el vector Xwnormal contiene ahora las amplitudes de las componentes de frecuencia de la señal Xw normalizadas de acuerdo con su media y desviación estándar

In [ ]:
#desviacion tarin=desvia_tra
mediatrain=np.mean(Xw)
desvia_tra=np.std(Xw)
Xwnormal = (Xw-mediatrain)/desvia_tra
Xwnormal

Al igual que en el caso anterior normalizamos Xwest ,y el vector Xwnormal1 contiene ahora las amplitudes de las componentes de frecuencia de la señal Xwtest normalizadas de acuerdo con su media y desviación estándar

In [ ]:
#desviaciontest1=desvia1
mediatest=np.mean(Xwtest)
desvia1=np.std(Xwtest)
Xwnormal1 = (Xwtest-mediatest)/desvia1
Xwnormal1

y para finiquitar usamos  la función cdist de scipy.spatial.distance para calcular distancias entre dos conjuntos de puntos y asignar etiquetas basadas en la distancia mínima.y realizar una asignación de etiquetas a los elementos del conjunto de prueba Xtest en función de la distancia mínima a los elementos del conjunto de entrenamiento Xw, utilizando las etiquetas correspondientes en Ytrain.

In [ ]:
from scipy.spatial.distance import cdist

contar= []
for i in range(len(Xtest)):
  p=cdist(Xwtest,Xw)
  contar.append(Ytrain[np.argmin(p[i,])])


Y graficamos cuantos errores tenemos de cada clase

In [ ]:
lista1=labels_
lista2=[contar.count(1),contar.count(2),contar.count(3),contar.count(4),contar.count(5),contar.count(6),contar.count(7),contar.count(8),contar.count(9),contar.count(10)]
y_pos = np.arange(len(lista2))
plt.barh(y_pos, lista2, align='center', alpha=0.5)
plt.yticks(y_pos, lista1)
plt.xlabel('Numero de errores')

plt.title('Grafico de numero de errores de cada tipo')
plt.savefig('barras_horizontal.png')
plt.show()